# Описание проекта

В рамках проекта этого модуля необходимо научиться по результатам опроса, который авиакомпания проводит после полёта, предсказывать удовлетворённость пассажиров услугами авиакомпании и совершённым перелётом

После перелёта пассажир заполнял анкету, которая состоит из различных вопросов про условия на борту самолёта и качество предоставленных услуг. Последний вопрос в анкете был сформулирован таким образом: оцените свою удовлетворённость совершённым перелётом

Задача состоит в том, чтобы на основе того, как пассажир отвечал на остальные вопросы анкеты, предсказать, остался он удовлетворён полётом или нет. Данная задача является классической задачей бинарной классификации, и решить её предлагается с помощью модели логистической регрессии

### Описание таблицы data

Для предсказания удовлетворённости пассажира полётом используются следующие факторы (в скобках указаны их типы и возможные значения):
- Gender — пол пассажира (бинарный: Female — женский, Male — мужской)
- Customer Type — тип пассажира (категориальный: Loyal customer — лояльный, disloyal Customer — не лояльный)
- Age — возраст пассажира (численный)
- Type of Travel — цель поездки (бинарный: Personal Travel — личная поездка, Business Travel — рабочая поездка)
- Class — класс полёта (категориальный: Business — бизнес-класс, Eco — эконом, Eco Plus — эконом-плюс)
- Flight distance — длина перелёта (численный)
- Inflight wifi service — удовлетворённость сетью Wi-Fi на борту самолёта (численный: 1–5 или 0, если не оценивался)
- Departure/Arrival time convenient — удовлетворённость временем вылета/прилёта (численный: 1–5 или 0, если не оценивался)
- Ease of Online booking — удовлетворённость удобством онлайн-бронирования билетов (численный: 1–5 или 0, если не оценивался)
- Gate location — удовлетворённость расположением выходов на посадку (численный: 1–5 или 0, если не оценивался)
- Food and drink — удовлетворённость питанием на борту (численный: 1–5 или 0, если не оценивался)
- Online boarding — удовлетворённость удобством онлайн-регистрации на рейс (численный: 1–5 или 0, если не оценивался)
- Seat comfort — удовлетворённость удобством мест в самолёте (численный: 1–5 или 0, если не оценивался)
- Inflight entertainment — удовлетворённость уровнем развлечений на борту самолёта (численный: 1–5 или 0, если не оценивался)
- On-board service — удовлетворённость уровнем обслуживания на борту самолёта (численный: 1–5 или 0, если не оценивался)
- Leg room service — удовлетворённость местом для ног перед сиденьем (численный: 1–5 или 0, если не оценивался)
- Baggage handling — удовлетворённость обращением с багажом (численный: 1–5 или 0, если не оценивался)
- Checkin service — удовлетворённость обслуживанием на стойке регистрации (численный: 1–5 или 0, если не оценивался)
- Cleanliness — удовлетворённость чистотой в самолёте (численный: 1–5 или 0, если не оценивался).
- Departure Delay in Minutes — задержка отправления самолёта в минутах (численный: 1–5 или 0, если не оценивался)
- Arrival Delay in Minutes — задержка прибытия самолёта в минутах (численный: 1–5 или 0, если не оценивался)
  
Предсказываемая характеристика:

- satisfaction — удовлетворённость полётом (бинарный: False — пассажир остался не удовлетворён полётом, True — пассажир остался удовлетворён полётом)

Важно, что предсказывать классы False и True можно так же, как и классы −1 и 1: Python будет интерпретировать значения False и True как 0 и 1 соответственно

Если классы принимают значения 0 и 1, то для обучения модели логистической регрессии с точностью до нескольких коэффициентов используется ровно тот же самый алгоритм, который был рассмотрен в соответствующем модуле

При этом подобное представление классов обладает интересными свойствами. В частности, если усреднить в какой-то группе объектов значения их классов (каждое из которых может быть либо 0, либо 1), то результат будет равен доле объектов группы, которые принадлежат классу 1

In [1]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns

## Обработка данных

In [2]:
main_data = pd.read_csv('data.csv')
data = main_data.copy()

In [3]:
data.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,...,5,5,4,3,4,4,5,25,18.0,False
1,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,...,1,1,1,5,3,1,1,1,6.0,False
2,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,...,5,5,4,3,4,4,5,0,0.0,True
3,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,...,2,2,2,5,3,1,2,11,9.0,False
4,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,...,5,3,3,4,4,3,3,0,0.0,True


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129880 entries, 0 to 129879
Data columns (total 22 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Gender                             129880 non-null  object 
 1   Customer Type                      129880 non-null  object 
 2   Age                                129880 non-null  int64  
 3   Type of Travel                     129880 non-null  object 
 4   Class                              129880 non-null  object 
 5   Flight Distance                    129880 non-null  int64  
 6   Inflight wifi service              129880 non-null  int64  
 7   Departure/Arrival time convenient  129880 non-null  int64  
 8   Ease of Online booking             129880 non-null  int64  
 9   Gate location                      129880 non-null  int64  
 10  Food and drink                     129880 non-null  int64  
 11  Online boarding                    1298

In [5]:
data['Arrival Delay in Minutes'] = data['Arrival Delay in Minutes'].fillna(0)
data['Arrival Delay in Minutes'] = data['Arrival Delay in Minutes'].astype('int')
data['satisfaction'] = data['satisfaction'].astype('int')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129880 entries, 0 to 129879
Data columns (total 22 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   Gender                             129880 non-null  object
 1   Customer Type                      129880 non-null  object
 2   Age                                129880 non-null  int64 
 3   Type of Travel                     129880 non-null  object
 4   Class                              129880 non-null  object
 5   Flight Distance                    129880 non-null  int64 
 6   Inflight wifi service              129880 non-null  int64 
 7   Departure/Arrival time convenient  129880 non-null  int64 
 8   Ease of Online booking             129880 non-null  int64 
 9   Gate location                      129880 non-null  int64 
 10  Food and drink                     129880 non-null  int64 
 11  Online boarding                    129880 non-null  

In [7]:
data.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,...,5,5,4,3,4,4,5,25,18,0
1,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,...,1,1,1,5,3,1,1,1,6,0
2,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,...,5,5,4,3,4,4,5,0,0,1
3,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,...,2,2,2,5,3,1,2,11,9,0
4,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,...,5,3,3,4,4,3,3,0,0,1


In [8]:
data.max()

Gender                                            Male
Customer Type                        disloyal Customer
Age                                                 85
Type of Travel                         Personal Travel
Class                                         Eco Plus
Flight Distance                                   4983
Inflight wifi service                                5
Departure/Arrival time convenient                    5
Ease of Online booking                               5
Gate location                                        5
Food and drink                                       5
Online boarding                                      5
Seat comfort                                         5
Inflight entertainment                               5
On-board service                                     5
Leg room service                                     5
Baggage handling                                     5
Checkin service                                      5
Cleanlines

In [9]:
data.min()

Gender                                        Female
Customer Type                         Loyal Customer
Age                                                7
Type of Travel                       Business travel
Class                                       Business
Flight Distance                                   31
Inflight wifi service                              0
Departure/Arrival time convenient                  0
Ease of Online booking                             0
Gate location                                      0
Food and drink                                     0
Online boarding                                    0
Seat comfort                                       0
Inflight entertainment                             0
On-board service                                   0
Leg room service                                   0
Baggage handling                                   1
Checkin service                                    0
Cleanliness                                   

Данные без явных выбросов

Проведём целевое кодирование

In [10]:
data_columns = data.columns[:-1]
for column in data_columns:
    data_group = data.groupby(column).agg({'satisfaction':'mean'})
    column_dict = data_group['satisfaction'].to_dict()
    data[column] = data[column].map(column_dict).astype('float')
data['satisfaction'] = data['satisfaction'].map({0:-1, 1: 1})

In [11]:
data.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0.440115,0.478115,0.167494,0.101326,0.246414,0.333333,0.251825,0.388614,0.310104,0.498886,...,0.651336,0.650615,0.534584,0.276062,0.480636,0.460023,0.612472,0.410256,0.363539,-1
1,0.440115,0.239697,0.331529,0.583724,0.694434,0.365269,0.251825,0.444739,0.310104,0.347062,...,0.223325,0.141946,0.196659,0.614561,0.237979,0.239570,0.196963,0.444595,0.350322,-1
2,0.428975,0.478115,0.320695,0.583724,0.694434,0.311111,0.247215,0.444739,0.303484,0.463780,...,0.651336,0.650615,0.534584,0.276062,0.480636,0.460023,0.612472,0.459390,0.473792,1
3,0.428975,0.478115,0.331529,0.583724,0.694434,0.326733,0.247215,0.424709,0.734676,0.568040,...,0.226024,0.212673,0.255463,0.614561,0.237979,0.239570,0.212649,0.439847,0.351213,-1
4,0.440115,0.478115,0.283843,0.583724,0.694434,0.321608,0.251825,0.439673,0.310104,0.347062,...,0.651336,0.273154,0.318093,0.583096,0.480636,0.450794,0.433075,0.459390,0.473792,1


Рассмотрим корреляцию между столбцами

## Создание модели логистической регрессии

In [12]:
def logistic_summary_loss_gradient_solution(w, X, y):
    """
    Вычисляет градиент функции суммарных потерь в случае обучения модели логистической регрессии.
    
    Аргументы:
        w: Вектор весов модели логистической регрессии. Первая координата вектора соответствует
           свободному коэффициенту, последующие — весам факторов.
        X: Матрица значений факторов.
           Первый столбец матрицы содержит значения константного коэффициента, который для всех объектов равен 1.
           Колонке с индексом i в матрице соответствует вес с индексом i в векторе w.
        y: Вектор классов, которым в реальности принадлежат объекты из матрицы X.
           Объекту в i-ой строчке матрицы X соответствует значение с индексом i вектора y.
        
    Возвращаемое значение:
        Вектор градиента функции суммарных потерь.
        Каждая координата вектора должна быть округлена до 2 знаков после запятой.
    """
    s = 1 / (1 + np.exp(y * np.matmul(X, w))) # Вычисления значений сигмоиды для объектов
    
    l = np.matmul(- y * s, X) # Вычисление градиента 
    
    return l

In [13]:
def logistic_regression_solve_solution(data, factor_names, y_name, learning_rate, eps):
    """
    С помощью градиентного спуска строит модель логистической регрессии по переданному набору данных.
    
    Аргументы:
        data: Таблица с объектами обучающей выборки.
              Каждый объект описывается набором численных факторов. 
              В данных может быть представлено больше факторов, чем модель должна использовать для предсказания. 
              Искусственного константного фактора, который для всех объектов равен 1 и 
              который будет использоваться моделью для предсказания, в таблице нет.
        factor_names: Список названий факторов, которые модель должна использовать для предсказания.
        y_name: Название столбца таблицы, в котором для каждого объекта содержится значение предсказываемого класса.
                Класс может иметь значение либо -1, либо 1.
        learning_rate: Опциональный параметр. Коэффициент скорости обучения, который используется в алгоритме градиентного спуска.
        eps: Опциональный параметр. Минимальное расстояние между текущей точкой градиентного спуска и следующей,
             при котором работа алгоритма останавливается.
        
    Возвращаемое значение:
        Возвращает вектор весов модели. 
        Координата вектора с индексом 0 соответствует свободному коэффициенту модели.
        Координата вектора с индексом i соответствует фактору с индексом i - 1 в списке factor_names.
    """
    X = np.hstack([np.ones([len(data), 1]), data[factor_names]])
    y = np.array(data[y_name])
    w = np.zeros(len(X[0]))
    
    while True:
        grad = logistic_summary_loss_gradient_solution(w, X, y) / len(y)
        w_ = w - learning_rate * grad
        if np.linalg.norm(w_ - w) <= eps:
            break
        w = w_
    
    return w_

In [14]:
factor_names = data.columns[:-1]
y_name = 'satisfaction'

Будем разбивать выборку на обучающую и тестовую в соотношении 3 к 1

In [15]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(data, data[y_name], test_size = 1/4)

In [16]:
w = logistic_regression_solve_solution(X_train, factor_names, y_name, 0.1, 0.01)

In [17]:
w

array([-0.42839102, -0.1858311 , -0.11338334, -0.01254932,  0.21483263,
        0.31122417,  0.09391507,  0.35424762, -0.17609253,  0.00324425,
       -0.1404744 , -0.08670727,  0.54825918,  0.1052532 ,  0.16621104,
        0.03743488,  0.05028909, -0.0177924 , -0.05996733,  0.0094777 ,
       -0.16681974, -0.15289128])

In [18]:
def logistic_regression_predict_solution(w, data, factor_names):
    """
    На основе переданного вектора весов для каждого объекта из
    набора данных делает предсказание с помощью модели логистической регрессии.
    
    Аргументы:
        w: Вектор весов модели логистической регрессии. Первая координата вектора соответствует
           свободному коэффициенту, последующие — весам факторов.
        data: Таблица с объектами, для которых необходимо сделать предсказания.
              Каждый объект описывается набором численных факторов. 
              В данных может быть представлено больше факторов, чем модель использует для предсказания. 
              Искусственного константного фактора, который для всех объектов равен `1` и 
              который используется моделью для предсказания, в таблице нет.
        factor_names: Список названий факторов, которые используются для предсказания. 
                      Порядок названий совпадает с порядком, в котором идут коэффициенты факторов
                      в векторе весов `w`.
        
    Возвращаемое значение:
        Вектор предсказанных вероятностей принадлежности положительному классу для объектов из переданной таблицы.
        Значения в векторе должны быть округлены до 2 знаков после запятой.
    """
    data_f = data[factor_names] # Оставляет в таблице только нужные факторы
    array_o = np.ones((len(data_f), 1)) # Создаёт единичный вектор-столбец
    array_f = np.hstack((array_o, np.array(data_f))) # Добавляет единичный вектор-столбец к таблице
    w_x = np.matmul(array_f, w) # Перемножаем таблицу факторово с вектором весов
    p = 1 / (1 + np.exp(- w_x)) # Вычисляет значения сигмоиды
    
    return p

In [19]:
probability = logistic_regression_predict_solution(w, X_test, factor_names)

## Проверка точности модели

In [20]:
probability

array([0.42548994, 0.59538941, 0.4048292 , ..., 0.41451448, 0.38941761,
       0.43590947], shape=(32470,))

In [21]:
y_test

72687    -1
87568     1
33684    -1
92979    -1
41538    -1
         ..
58679     1
95098     1
73990    -1
123354   -1
123636   -1
Name: satisfaction, Length: 32470, dtype: int64

In [22]:
y_p = {
    'y': y_test,
    'p': probability
}

In [23]:
y_p = pd.DataFrame(y_p)

In [24]:
y_p.head()

,y,p
72687,-1,0.425490
87568,1,0.595389
33684,-1,0.404829
92979,-1,0.393604
41538,-1,0.388493


In [25]:
y_p['y_'] = y_p['p'].apply(lambda x: 1 if x > 0.5 else -1)

In [26]:
y_p['acc'] = y_p['y'] * y_p['y_']

In [27]:
y_p['acc'] = y_p['acc'].replace({1: 1, -1: 0})

In [28]:
accuracy = y_p['acc'].mean()

In [29]:
accuracy

np.float64(0.8759162303664921)

Точность 88 % по метрике accuracy

Проверим баланс классов

In [30]:
TP = y_p['y'][(y_p['y'] == 1) & (y_p['y_'] == 1)].count()
FN = y_p['y'][(y_p['y'] == 1) & (y_p['y_'] == -1)].count()
FP = y_p['y'][(y_p['y'] == -1) & (y_p['y_'] == 1)].count()
TN = y_p['y'][(y_p['y'] == -1) & (y_p['y_'] == -1)].count()

In [31]:
TP, FN, FP, TN

(np.int64(11797), np.int64(2235), np.int64(1794), np.int64(16644))

In [32]:
precision = TP / (TP + FP)
recall = TP / (TP + FN)

In [33]:
precision

np.float64(0.8680008829372379)

In [34]:
recall

np.float64(0.8407212086659065)

In [35]:
F1_score = 2 * TP / (2 * TP + FP + FN)

In [36]:
F1_score

np.float64(0.854143286391775)

Основные метрики показали точность не менее 84 %